In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
# data
import tensorflow as tf

## 데이터 불러오기

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

cifar100_train = [(torch.Tensor(x), y) for x,y in zip(x_train.transpose(0,3,1,2), y_train.flatten())]
cifar100_test = [(torch.Tensor(x), y) for x,y in zip(x_test.transpose(0,3,1,2), y_test.flatten())]

In [0]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

## Dataloader 생성

In [0]:
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
    
NUM_TRAIN = 49000
NUM_VAL = 1000

loader_train = DataLoader(cifar100_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))
loader_val = DataLoader(cifar100_train, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))
loader_test = DataLoader(cifar100_test, batch_size=64)

## CUDA 여부 확인 False 일 경우 설정 변경

In [0]:
torch.cuda.is_available()

True

In [0]:
dtype = torch.cuda.FloatTensor

## Train 및 Acc 함수 들 필요시 수정 가능 !

In [0]:
def train(model, loss_fn, optimizer, num_epochs, dtype):
    for epoch in range(num_epochs):
#         print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        total_loss = 0
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.type(dtype))
            y_var = Variable(y.type(dtype).long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            total_loss += loss.data

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (epoch+1) % 10 == 0:
            print(epoch+1)
            print('loss = %.4f' % (total_loss/(t+1)))
            print('train :')
            check_accuracy(model, loader_val, dtype)

def check_accuracy(model, loader, dtype):
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    with torch.no_grad():
        for x, y in loader:
            x_var = Variable(x.type(dtype))

            scores = model(x_var)
            _, preds = scores.data.cpu().max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    

## 나만의 모델 생성

In [0]:
class AlexNet(nn.Module):
    def __init__(self, num_classes, dtype):
        super(AlexNet, self).__init__()
        self.dtype = dtype
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        ).type(dtype)
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),
            nn.Linear(512, 512),
            nn.Linear(512, num_classes)
        ).type(dtype)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [0]:
alexnet = AlexNet(100, dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(alexnet.parameters(),lr=1e-4)

## Train and check

In [15]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(5, 5))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=100, bias=

In [11]:
torch.cuda.random.manual_seed(12345)
alexnet.apply(reset)
train(alexnet, loss_fn, optimizer, 100, dtype)

10
loss = 1.8545
train :
Got 275 / 1000 correct (27.50)
20
loss = 0.4440
train :
Got 275 / 1000 correct (27.50)
30
loss = 0.2101
train :
Got 281 / 1000 correct (28.10)
40
loss = 0.1686
train :
Got 282 / 1000 correct (28.20)
50
loss = 0.1275
train :
Got 291 / 1000 correct (29.10)
60
loss = 0.1080
train :
Got 276 / 1000 correct (27.60)
70
loss = 0.1048
train :
Got 279 / 1000 correct (27.90)
80
loss = 0.1042
train :
Got 288 / 1000 correct (28.80)
90
loss = 0.0961
train :
Got 278 / 1000 correct (27.80)
100
loss = 0.0985
train :
Got 292 / 1000 correct (29.20)


In [0]:
check_accuracy(alexnet, loader_test, dtype)

Got 2897 / 10000 correct (28.97)
